In [19]:
import pandas as pd
import numpy as np
from math import sqrt

In [41]:
N =12

In [20]:
#Load csv files 
df_articles = pd.read_csv('articles.csv')
df_transactions = pd.read_csv('transactions_train.csv')
df_customers = pd.read_csv('customers.csv')

In [21]:
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])

In [22]:
df = df_trans[['t_dat', 'customer_id', 'article_id']].copy()
#Calculate week for each date
df['week'] = 104 - (df['t_dat'].max() - df['t_dat']).dt.days // 7  #104 is the maximum week number 
 #Weekly sale count of each articles
weekly_sales = df.drop('customer_id', axis=1).groupby(['week', 'article_id']).size().reset_index().rename(columns={0: 'count'})
 #calculate week and article id wise purchase coun
df = df.merge(weekly_sales, on=['week', 'article_id'], how = 'left')
df = df.merge(weekly_sales.loc[weekly_sales['week']==104, ['count', 'article_id']], on='article_id', suffixes=("", "_target"))
df['count_target'].fillna(0, inplace=True) 
del weekly_sales
#Normalize article counts with respect to week to remove bias and seasonlity 
df['norm_count'] = df['count_target'] / df['count']

In [27]:
#purchase dict contain customer article value dict
customer_article_value_dict = {}
last_ts = df['t_dat'].max()
#Looping for each customer
for i in df.index:
    cust_id = df.at[i, 'customer_id']
    art_id = df.at[i, 'article_id']
    t_dat = df.at[i, 't_dat']

    if cust_id not in customer_article_value_dict:
        customer_article_value_dict[cust_id] = {}

    if art_id not in customer_article_value_dict[cust_id]:
        customer_article_value_dict[cust_id][art_id] = 0
    
    #Assigning weight according to day from maximum purchase date
    days_from_maximum_date = max(1, (last_ts - t_dat).days)
    weight = 250000 / np.sqrt(days_from_maximum_date) + 100000 * np.exp(-0.2*days_from_maximum_date) - 1000
    value = df.at[i, 'norm_count'] * max(0, weight)
    #Adding the value for article on the cystomer
    customer_article_value_dict[cust_id][art_id] += value

target_sales = df.drop('customer_id', axis=1).groupby('article_id')['norm_count'].sum()
default_pred = target_sales.nlargest(N).index.tolist()

In [29]:
sub = pd.read_csv('sample_submission.csv')

In [36]:
pred_list = []

In [37]:
#Looping for each customer
for cust_id in sub['customer_id']:
    #Previous customer then use history otherwise assign default prediction
    if cust_id in customer_article_value_dict:
        series = pd.Series(customer_article_value_dict[cust_id])
        #Getting top n accordng to value
        tempTop = series.nlargest(N).index.tolist()
        tmp_l = tempTop.copy()
        if len(tmp_l) < N:
            tempTop = tmp_l + default_pred[:(N-len(tempTop))]
    else:
        tempTop = default_pred
    pred_list.append(' '.join(tempTop))
sub['prediction'] = pred_list

In [38]:
sub.shape

(1371980, 2)

In [39]:
sub.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0656719005 0176209023 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0800436010 0673677002 0739590027 06...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0852643003 0858883002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0448509014 0573085028 0751471001 0706016001 06...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0818320001 09...


In [40]:
sub.to_csv('submission_trending.csv',index=False)